In [153]:
import tensorflow as tf
from skimage import color, exposure, transform
import numpy, os
import glob, os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tensorflow.contrib.layers import flatten
from sklearn.utils import shuffle

In [452]:


def preprocess_img(img):
    IMG_SIZE = 32
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE),mode='constant')

    
    return img


def get_class(img_path):
    return int(img_path.split('/')[-2])

## root_dir :  Need to be a PATH with /(slash) not \ (backslash) example: C:/Users/Usuario/Documents/German-Traffic-Signs-Detector/images/FullIJCNN2013
def readImageAndProccess(root_dir):
    imgs = []
    labels = []

    all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
    ##print(all_img_paths)
    ##np.random.shuffle(all_img_paths)
    ##print(all_img_paths)
    for img_path in all_img_paths:
        ##print(img_path)
        img_path2=img_path.replace("\\","/")
        ##print(img_path2) io.imread                          
        img = preprocess_img(cv2.imread(img_path))
        label = get_class(img_path2)
        imgs.append(img)
        labels.append(label)
    ##print(labels)    
    ##print(X2.shape)
    ##print(nX2.shape)
    # Make one hot targets
    ##Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
    ##print(len(labels))
    return imgs,labels

In [315]:
image_size=32
num_channels=3
num_labels=43
path="C:/Users/Usuario/Documents/German-Traffic-Signs-Detector/images/FullIJCNN2013"
imgs,labels=readImageAndProccess(path)
imgs_nparray = np.array(imgs, dtype='float32')
X = np.reshape(imgs_nparray, (imgs_nparray.shape[0], -1))
NUM_CLASSES = 43
Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]



In [440]:
dataset = X.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
print(dataset[3].shape)
print(Y.shape)
x_train, x_test, y_train, y_test = train_test_split(dataset,Y, test_size=0.20, random_state=0)


(32, 32, 3)
(1213, 43)


In [441]:
print('Training set', x_train.shape, y_train.shape)
print('Test set', x_test.shape, y_test.shape)



Training set (970, 32, 32, 3) (970, 43)
Test set (243, 32, 32, 3) (243, 43)


In [442]:
EPOCHS = 50
BATCH_SIZE = 128

In [443]:
def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
        'layer_1' : 6,
        'layer_2' : 16,
        'layer_3' : 120,
        'layer_f1' : 84
    }

    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,3,6],mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = flatten(pool_2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)
    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,43), mean = mu , stddev = sigma))
    fc3_b = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_w) + fc3_b
    return logits

In [444]:
X_ =  tf.placeholder (tf.float32, (None, 32, 32, 3))
Y_ = tf.placeholder(tf.float32, [None, 43])



In [445]:
rate = 0.001

In [446]:
logits=LeNet(X_)

In [447]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_)

In [448]:
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [449]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [450]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={X_: batch_x, Y_: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [451]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(x_train)

    for i in range(EPOCHS):
        x_train, y_train = shuffle(x_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = x_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={X_: batch_x, Y_: batch_y})
            
        validation_accuracy = evaluate(x_test, y_test)
       
    print ("Accuracy:", accuracy_operation.eval({X_: x_test, Y_: y_test}))
    sess.close()

Accuracy: 0.89711934
